In [1]:
import urllib.request 
import os
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.linear_model import LinearRegression
import datetime

In [2]:
## Download and unzip COVID cases
temporary_folder = os.path.join(os.getcwd(), 'tmp')
if not os.path.exists(temporary_folder):
    os.makedirs(temporary_folder)
    
remote_file_name = "https://github.com/CSSEGISandData/COVID-19/archive/master.zip"
base_name = os.path.basename(remote_file_name)
local_file_name = os.path.join(temporary_folder, base_name)

urllib.request.urlretrieve(remote_file_name, local_file_name)

data_folder = os.path.join(os.getcwd(), 'world_data') 
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
    
with zipfile.ZipFile(local_file_name, 'r') as zip_ref:
    zip_ref.extractall(data_folder)
    
# !unzip master.zip

In [5]:
## Download world population
remote_file_name = "http://api.worldbank.org/v2/en/indicator/SP.POP.TOTL?downloadformat=excel"
world_population_file = os.path.join(temporary_folder, "world_population.xls")

# The following statement is throwing an alert but the download is being performed anyway
urllib.request.urlretrieve(remote_file_name, world_population_file) 

('/home/tassio/Desenvolvimento/python/COVID-19/tmp/world_population.xls',
 <http.client.HTTPMessage at 0x7f9074438e20>)

In [8]:
## Load world population DataFrame
print(world_population_file)
# !pip install xlrd

world_population = pd.read_excel(world_population_file, sheet_name="Data", dtype=object, skiprows=3)
world_population = world_population[["Country Name", "Country Code", "2018"]]
world_population = world_population.rename(
    {"Country Name":"country","Country Code":"country_code", "2018":"population"}, 
    axis='columns') 

world_population['population'] = world_population['population'].fillna(0)
world_population['population'] = world_population['population'].astype('int')

/home/tassio/Desenvolvimento/python/COVID-19/tmp/world_population.xls


In [13]:
## Load COVID cases and standardizing the datasets
covid_folder = os.path.join(data_folder, 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/')

li = []
file_list = os.listdir(covid_folder)
file_list.sort()

for covid_file in file_list:
    if covid_file.find(".csv") > -1:
        df = pd.read_csv(os.path.join(covid_folder, covid_file), index_col=None, header=0)
        if "Province/State" in df:
            df = df.rename({"Province/State":"province_state"}, axis="columns")
        elif "Province_State" in df:
            df = df.rename({"Province_State":"province_state"}, axis="columns")
  
        if "Country/Region" in df:
            df = df.rename({"Country/Region":"country"}, axis="columns")
        elif "Country_Region" in df:
            df = df.rename({"Country_Region":"country"}, axis="columns")

        if "Last Update" in df:
            df = df.rename({"Last Update":"last_update"}, axis="columns")
        elif "Last_Update" in df:
            df = df.rename({"Last_Update":"last_update"}, axis="columns")
            
        if "Confirmed" in df:
            df = df.rename({"Confirmed":"confirmed"}, axis="columns")
            
        if "Deaths" in df:
            df = df.rename({"Deaths":"deaths"}, axis="columns")
            
        df = df[["province_state", "country", "last_update", "confirmed", "deaths"]]
        li.append(df)

covid_cases = pd.concat(li, axis=0, ignore_index=True)
covid_cases = covid_cases.rename(
   {
       "Confirmed":"confirmed",
       "Deaths":"deaths",
       "Recovered":"recovered",
       "Latitude":"latitude",
       "Longitude":"longitude"
   }, axis='columns'
) 

print(covid_cases)
print(covid_cases[covid_cases["country"] == "Italy"])
# print(recent)
# covid = pd.read_csv(covid)
# print(covid)

province_state             country          last_update  confirmed  \
0              Anhui      Mainland China      1/22/2020 17:00        1.0   
1            Beijing      Mainland China      1/22/2020 17:00       14.0   
2          Chongqing      Mainland China      1/22/2020 17:00        6.0   
3             Fujian      Mainland China      1/22/2020 17:00        1.0   
4              Gansu      Mainland China      1/22/2020 17:00        NaN   
...              ...                 ...                  ...        ...   
24702            NaN           Venezuela  2020-03-26 23:48:18      107.0   
24703            NaN             Vietnam  2020-03-26 23:48:18      153.0   
24704            NaN  West Bank and Gaza  2020-03-26 23:48:18       84.0   
24705            NaN              Zambia  2020-03-26 23:48:18       16.0   
24706            NaN            Zimbabwe  2020-03-26 23:48:18        3.0   

       deaths  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN

In [14]:
## Adjust country names and merging dataframes
world_population = world_population.replace("Korea, Rep.", "South Korea")
world_population = world_population.replace("Hong Kong SAR, China", "Hong Kong")
world_population = world_population.replace("Iran, Islamic Rep.", "Iran")
world_population = world_population.replace("Russian Federation", "Russia")
world_population = world_population.replace("Macao SAR, China", "Macau")
world_population = world_population.replace("Venezuela, RB", "Venezuela")
world_population = world_population.replace("Bahamas, The", "The Bahamas")
world_population = world_population.replace("Egypt, Arab Rep.", "Egypt")
world_population = world_population.replace("Slovak Republic", "Slovakia")
covid_cases = covid_cases.replace("Mainland China", "China")
covid_cases = covid_cases.replace("US", "United States")
covid_cases = covid_cases.replace("UK", "United Kingdom")
covid_cases = covid_cases.replace("Iran (Islamic Republic of)", "Iran")
covid_cases = covid_cases.replace("Russian Federation", "Russia")
covid_cases = covid_cases.replace("Republic of Korea", "South Korea")
covid_cases = covid_cases.replace("Hong Kong SAR", "Hong Kong")
covid_cases = covid_cases.replace("Viet Nam", "Vietnam")
covid_cases = covid_cases.replace("Macao SAR", "Macau")
covid_cases = covid_cases.replace("Korea, South", "South Korea")
covid_cases = covid_cases.replace("North Ireland", "United Kingdom")
covid_cases = covid_cases.replace("Republic of Ireland", "Ireland")

covid_cases_pop = pd.merge(covid_cases, world_population, how='left', on='country')

# The following line was used to check countries on population dataset and not on casesdataset and vice-versa
# print(((covid_cases_pop[covid_cases_pop['population'].isnull()])["country"]).drop_duplicates())

In [15]:
## Remove countries with no population 
covid_cases_pop['population'] = covid_cases_pop['population'].fillna(0)
covid_cases_pop = covid_cases_pop.loc[covid_cases_pop['population'] != 0]
covid_cases_pop['population'] = covid_cases_pop['population'].astype('int')
covid_cases_pop["last_update"] = pd.to_datetime(covid_cases_pop["last_update"])
covid_cases_pop["last_update"] = covid_cases_pop["last_update"].dt.date

covid_cases_aggr = covid_cases_pop[["country", "last_update", "population", "confirmed", "deaths"]]

In [16]:
## Aggregate by country, date and population
covid_cases_aggr = covid_cases_aggr.groupby(["country", "last_update", "population"]).sum()
covid_cases_aggr = covid_cases_aggr.reset_index()

In [17]:
## Generate forecast for N days (set on days_to_forecast variable) beyond the last data point
## It uses an exponential regresion as a model to forecast
days_to_forecast = 10

countries = covid_cases_aggr[covid_cases_aggr["country"].isin(["Brazil", "Italy", "United States", "South Korea"])]
countries = countries["country"].drop_duplicates()

li = []

for country in countries:
    country_cases = covid_cases_aggr[covid_cases_aggr["country"] == country]
    country_cases = country_cases[country_cases["confirmed"] > 0]
    country_dates = pd.date_range(start=country_cases.last_update.min(), end=covid_cases_aggr.last_update.max())
    country_cases = country_cases.set_index('last_update').reindex(country_dates).fillna(method='pad').rename_axis('last_update').reset_index()
    country_cases = country_cases.reset_index()
    country_cases.rename(columns = {'index':'days_since_first_case'}, inplace = True) 
    country_cases["type"] = "actuals"
    x = np.array((country_cases["days_since_first_case"]))
    y = np.array((country_cases["confirmed"]))
    confirmed_cases_pred = np.polyfit(x, np.log(y), 1)
    
    population = int((country_cases["population"]).tail(1))
    death_rate = float((country_cases["deaths"].tail(1) * 1.0000) / (country_cases["confirmed"].tail(1) * 1.0000))
#    for i in range(country_cases["days_since_first_case"].max() + 1 , country_cases["days_since_first_case"].max() + days_to_forecast - 1):
    for i in range(1 , country_cases["days_since_first_case"].max() + days_to_forecast - 1):
        confirmed = np.exp(confirmed_cases_pred[1]) * np.exp(confirmed_cases_pred[0] * i) 
        deaths = confirmed * death_rate
        row = {
            "days_since_first_case": i,
            "last_update": (country_cases["last_update"].min() + datetime.timedelta(days=i)),
            "country": country,
            "population": population,
            "confirmed": round(confirmed),
            "deaths": round(deaths),
            "type": "forecast"
        }
        country_cases = country_cases.append(row, ignore_index=True)
    li.append(country_cases)

covid_cases_final = pd.concat(li, axis=0, ignore_index=True)

In [18]:
## Add rate columns
covid_cases_final["cases_per_million"] = covid_cases_final["confirmed"] / (covid_cases_final["population"] / 1000000)
covid_cases_final["fatalities_per_million"] = covid_cases_final["deaths"] / (covid_cases_final["population"] / 1000000)
covid_cases_final["fatalities_per_cases"] = covid_cases_final["deaths"] / covid_cases_final["confirmed"]

In [2]:
## Generate plot for Brazil
plt.figure(num=None, figsize=(18, 10), dpi=640, facecolor='w', edgecolor='k')
plt.xlabel('Days Since First Confirmed Case')
plt.ylabel('Cases Per Million')
plt.title('COVID-19 Country Curves Comparison')

comparing_country_legend = "Brazil"
comparing_country = covid_cases_final[covid_cases_final["country"] == comparing_country_legend]
point_in_time = comparing_country["days_since_first_case"].max()
green_legend = mpatches.Patch(color='green', label=comparing_country_legend)
green_legend_forecast = mpatches.Patch(color='cyan', label=comparing_country_legend + " (forecast)", linestyle = ':')
plt.plot('days_since_first_case', 'cases_per_million', data=comparing_country[comparing_country["type"] == "actuals"], linewidth=3, color="green")
plt.plot('days_since_first_case', 'cases_per_million', data=comparing_country[comparing_country["type"] == "forecast"], linewidth=3, color="cyan", linestyle = ':')

comparing_country = "Italy"
red_legend = mpatches.Patch(color='red', label=comparing_country)
comparing_country = covid_cases_final[covid_cases_final["country"] == comparing_country]
comparing_country = comparing_country[comparing_country["days_since_first_case"] <= point_in_time]
plt.plot('days_since_first_case', 'cases_per_million', data=comparing_country[comparing_country["type"] == "actuals"], linewidth=3, color="red")

comparing_country = "United States"
blue_legend = mpatches.Patch(color='blue', label=comparing_country)
comparing_country = covid_cases_final[covid_cases_final["country"] == comparing_country]
comparing_country = comparing_country[comparing_country["days_since_first_case"] <= point_in_time]
plt.plot('days_since_first_case', 'cases_per_million', data=comparing_country[comparing_country["type"] == "actuals"], linewidth=3, color="blue")

comparing_country = "South Korea"
black_legend = mpatches.Patch(color='black', label=comparing_country)
comparing_country = covid_cases_final[covid_cases_final["country"] == comparing_country]
comparing_country = comparing_country[comparing_country["days_since_first_case"] <= point_in_time]
plt.plot('days_since_first_case', 'cases_per_million', data=comparing_country[comparing_country["type"] == "actuals"], linewidth=3, color="black")

plt.legend(handles=[green_legend, green_legend_forecast, red_legend, blue_legend, black_legend])

plt.savefig('imagens/curva_brasil_italia_US_KS.png', format='png', dpi=100)

plt.show()

NameError: name 'plt' is not defined

In [20]:
## Generate plot for USA
plt.figure(num=None, figsize=(18, 10), dpi=640, facecolor='w', edgecolor='k')
plt.xlabel('Days Since First Confirmed Case')
plt.ylabel('Cases Per Million')
plt.title('COVID-19 Country Curves Comparison')

comparing_country_legend = "United States"
comparing_country = covid_cases_final[covid_cases_final["country"] == comparing_country_legend]
point_in_time = comparing_country["days_since_first_case"].max()
green_legend = mpatches.Patch(color='green', label=comparing_country_legend)
green_legend_forecast = mpatches.Patch(color='cyan', label=comparing_country_legend + " (forecast)", linestyle = ':')
plt.plot('days_since_first_case', 'cases_per_million', data=comparing_country[comparing_country["type"] == "actuals"], linewidth=3, color="green")
plt.plot('days_since_first_case', 'cases_per_million', data=comparing_country[comparing_country["type"] == "forecast"], linewidth=3, color="cyan", linestyle = ':')

comparing_country = "Italy"
red_legend = mpatches.Patch(color='red', label=comparing_country)
comparing_country = covid_cases_final[covid_cases_final["country"] == comparing_country]
comparing_country = comparing_country[comparing_country["days_since_first_case"] <= point_in_time]
plt.plot('days_since_first_case', 'cases_per_million', data=comparing_country[comparing_country["type"] == "actuals"], linewidth=3, color="red")

comparing_country = "South Korea"
blue_legend = mpatches.Patch(color='blue', label=comparing_country)
comparing_country = covid_cases_final[covid_cases_final["country"] == comparing_country]
comparing_country = comparing_country[comparing_country["days_since_first_case"] <= point_in_time]
plt.plot('days_since_first_case', 'cases_per_million', data=comparing_country[comparing_country["type"] == "actuals"], linewidth=3, color="blue")

comparing_country = "Brazil"
black_legend = mpatches.Patch(color='black', label=comparing_country)
comparing_country = covid_cases_final[covid_cases_final["country"] == comparing_country]
comparing_country = comparing_country[comparing_country["days_since_first_case"] <= point_in_time]
plt.plot('days_since_first_case', 'cases_per_million', data=comparing_country[comparing_country["type"] == "actuals"], linewidth=3, color="black")

plt.legend(handles=[green_legend, green_legend_forecast, red_legend, blue_legend, black_legend])

plt.savefig('imagens/curva_brasil_italia_US_KS_day_confirm.png', format='png', dpi=100)

plt.show()